# **Final Project Task 1 - Census Data Preprocess**

Requirements

- Target variable specification:
    - The target variable for this project is hours-per-week. 
    - Ensure all preprocessing steps are designed to support regression analysis on this target variable.
- Encode data  **3p**
- Handle missing values if any **1p**
- Correct errors, inconsistencies, remove duplicates if any **1p**
- Outlier detection and treatment if any **1p**
- Normalization / Standardization if necesarry **1p**
- Feature engineering **3p**
- Train test split, save it.
- Others?


Deliverable:

- Notebook code with no errors.
- Preprocessed data as csv.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

data = pd.read_csv(data_url, header=None, names=columns, na_values=" ?", skipinitialspace=True)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# 1) Corectarea valorilor lipsă mascate
# În Adult dataset, unele valori lipsă sunt codificate ca "?" sau " ?"
# Le transformăm explicit în NaN pentru a putea fi tratate uniform
data = data.replace("?", np.nan).replace(" ?", np.nan)

# 2) Eliminarea observațiilor duplicate (dacă există)
data = data.drop_duplicates()

# 3) Tratarea valorilor lipsă
# Pentru variabilele categorice cheie, imputăm valorile lipsă cu moda
for col in ["workclass", "occupation", "native-country"]:
    if data[col].isna().any():
        data[col] = data[col].fillna(data[col].mode(dropna=True)[0])

# 4) Feature Engineering
# 4.1) Crearea unei variabile care reprezintă câștigul net din capital
data["capital-net"] = data["capital-gain"] - data["capital-loss"]

# 4.2) Crearea unei variabile binare care indică statutul marital (căsătorit / necăsătorit)
data["is_married"] = data["marital-status"].astype(str).apply(
    lambda x: 1 if "Married" in x else 0
)

# Eliminăm variabila marital-status pentru a evita redundanța informațională
data = data.drop(columns=["marital-status"])

# 5) Tratarea valorilor extreme (outliers) folosind regula IQR
def cap_outliers_iqr(serie):
    q1, q3 = serie.quantile([0.25, 0.75])
    iqr = q3 - q1
    limita_inferioara = q1 - 1.5 * iqr
    limita_superioara = q3 + 1.5 * iqr
    return serie.clip(lower=limita_inferioara, upper=limita_superioara)

# Aplicăm limitarea valorilor extreme pe variabilele cu distribuții foarte asimetrice
for col in ["capital-gain", "capital-loss", "capital-net"]:
    data[col] = cap_outliers_iqr(data[col])

# 6) Codificarea variabilelor
# 6.1) Codificare binară pentru sex
data["sex"] = data["sex"].astype(str).str.strip().map({"Male": 1, "Female": 0})

# 6.2) Codificare binară pentru venit
# Eliminăm eventuale caractere speciale (ex. punct) și mapăm numeric
data["income"] = data["income"].astype(str).str.strip().str.replace(".", "", regex=False)
data["income"] = data["income"].map({"<=50K": 0, ">50K": 1})

# 6.3) One-Hot Encoding pentru variabilele categorice nominale
categorical_cols = [
    "workclass", "education", "occupation",
    "relationship", "race", "native-country"
]

data_final = pd.get_dummies(
    data,
    columns=categorical_cols,
    drop_first=True
)

# 7) Definirea variabilei țintă pentru regresie
TARGET = "hours-per-week"
X = data_final.drop(TARGET, axis=1)
y = data_final[TARGET]

# Asigurare finală: nu există valori NaN în matricea de feature-uri
X = X.fillna(0)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 10) Împărțirea în set de antrenare și set de testare
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 11) Standardizare (normalizare tip z-score)
# Important: fit doar pe train, transform pe test (evităm scurgeri de informație)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 12) Construirea dataseturilor finale pentru export (feature-uri + target)
train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
train_df["target_hours"] = y_train.values

test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
test_df["target_hours"] = y_test.values

# 13) Salvarea livrabilelor ca CSV (train și test)
train_df.to_csv("preprocessed_census_train.csv", index=False)
test_df.to_csv("preprocessed_census_test.csv", index=False)

# 14) Verificări finale (pentru a evita warnings/erori ulterioare)
print("Dimensiuni TRAIN:", train_df.shape, "| Dimensiuni TEST:", test_df.shape)
print("NaN în TRAIN:", train_df.isna().sum().sum(), "| NaN în TEST:", test_df.isna().sum().sum())
print("Fișierele preprocesate au fost salvate cu succes: preprocessed_census_train.csv și preprocessed_census_test.csv")


Dimensiuni TRAIN: (26029, 94) | Dimensiuni TEST: (6508, 94)
NaN în TRAIN: 0 | NaN în TEST: 0
Fișierele preprocesate au fost salvate cu succes: preprocessed_census_train.csv și preprocessed_census_test.csv
